# Analysing predictions UD POS different languages

In [1]:
import pandas as pd
import utils
from collections import defaultdict

pd.options.display.max_rows = 100000
pd.set_option('display.max_colwidth', -1)

PATH="C:/Users/bdolicki/Documents/Git/multilingual-analysis/code/analysing_predictions/udpos"

C:\Users\bdolicki\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [2]:
# we use a sample of 3 languages as an example, but you can use any language that is included in all_langs
all_langs = ['af', 'ar', 'bg', 'el', 'en', 'es', 'et', 'eu', 'fa', 'fi', 'fr', 'he', 'hi', 'id', 'it', 'ja', 'ko', 'nl', 'pt', 'ru']
train_langs = ['af', 'ar', 'en']
test_langs = ['af', 'ar', 'en']

## Aggregated analysis

### Helper functions
The functions below can be used to observe different aspects of predictions. We include them all in our script to show an example usage, but, depending what we focus on, they can be used rather independently - for example one can use only the first function.

In [3]:
# show most frequently confused POS tags for a language pair, conf is the confusion matrix
def most_confused_pos(conf):
    conf_count = pd.DataFrame()
    conf_count["count"] = conf.sum().sort_values(ascending=False) 
    conf_count["percent"] = conf_count/conf_count.sum()
    print("Most confused POS tags")
    display(conf_count[:5])
    return conf_count
    
# show most frequently confused POS pairs for a language pair, conf is the confusion matrix
def most_confused_pos_pairs(conf):
    sparse_conf_df = conf.reset_index().melt(id_vars="index",var_name="actual",value_name="count").rename(columns={"index":"pred"})
    sparse_conf_df["percent"] = sparse_conf_df["count"]/sparse_conf_df["count"].sum()
    sparse_conf_df = sparse_conf_df.sort_values(by="count",ascending=False)
    print("Most confused POS pairs")
    display(sparse_conf_df.head())

# show top 5 most frequently confused words and how many times each word was assigned a particular POS tag
# useful to see if a word has multiple POS tags or not
# mistakes is the df with mistaken words
def most_mistaken_word_with_tags(mistakes):
    for token in mistakes_df.token.value_counts()[:5].index:
        token_tags = defaultdict(int)
        for i,sentence in enumerate(pa.all_sentences):
            words = sentence.split()
            for j,word in enumerate(words):
                if word==token:
                    tag = pa.all_tags[i].split()[j]
                    token_tags[tag] += 1
        print("Possible tags for the word",token)
        print(dict(token_tags))
        
# show how many times a word was mistaken together with its frequency
def mistaken_words_with_word_freq(pa):
    def is_all(actual, pred=None):
        return True
    all_df, _ = pa.transform_data(is_all,conf_matrix=True)
    relative = pd.DataFrame()
    relative["% mistaken"] = mistakes_df.token.value_counts().divide(all_df.token.value_counts()).sort_values(ascending=False)
    relative["all_count"] = all_df.token.value_counts()
    relative["mistaken_count"] = mistakes_df.token.value_counts()
    relative = relative.dropna()
    print("Relative mistaken word frequency")
    display(relative.sort_values(by="% mistaken",ascending=False).head())

#   show most mistaken words per target language from all source languages
def most_mistaken_words_per_target_func(most_mistaken_words_list):
    print("Most mistaken words per target language from all source languages")
    display(pd.Series(most_mistaken_words_per_target).value_counts()[:5])

#   show most mistaken POS tags per target language from all source languages
def most_mistaken_pos_per_target_func(most_mistaken_pos_list):
    print("Most mistaken POS tags per target language from all source languages")
    display(pd.Series(most_mistaken_pos_per_target).value_counts()[:5])

#   show top 1 mistaken POS tag from all target languages
def top1_mistaken_pos_per_target(most_mistaken_pos_lang):
    print("Top 1 mistaken POS tag for all target languages")
    display(pd.DataFrame.from_dict(most_mistaken_pos_lang))

In [5]:
most_mistaken_pos_lang = {"lang":[],"tag":[]}
mistakes = defaultdict(dict)

for test_lang in test_langs:
#     aggregate most mistaken words/pos tags per target language for all source languages
    most_mistaken_words_per_target = []
    most_mistaken_pos_per_target = []
    print("test",test_lang)
    for train_lang in train_langs:
        print('train',train_lang)
        pa = utils.PredictionsAnalyzer()
        ud_pos = pa.read_tag_data(f"{PATH}/dev_sets/dev-{test_lang}.tsv",f"{PATH}/results/dev_{train_lang}_{test_lang}_predictions.txt")
        def is_pred_wrong(actual,pred=None):
            return actual!=pred

        mistakes_df, conf_df = pa.transform_data(is_pred_wrong,conf_matrix=True)
#         store df with mistakes for all language pairs if you want to perform further analysis
#         when running for a big subset of languages it might cause memory issues
        mistakes[train_lang][test_lang] = mistakes_df
        
        conf_count = most_confused_pos(conf_df)
        most_confused_pos_pairs(conf_df)
        most_mistaken_word_with_tags(mistakes_df)
        
        if train_lang==test_lang:
            print("Supervised")
        else:
            print("Zero-shot")

        most_mistaken_words_per_target += list(mistakes_df.token.value_counts()[:2].index)
        most_mistaken_pos_per_target += [conf_count.index[0]]
        
        mistaken_words_with_word_freq(pa)
        
    most_mistaken_words_per_target_func(most_mistaken_words_per_target)
    most_mistaken_pos_per_target_func(most_mistaken_pos_per_target)
    
    most_mistaken_pos_lang["lang"].append(test_lang)
    most_mistaken_pos_lang["tag"].append(pd.Series(most_mistaken_pos_per_target).value_counts().index[0])


test af
train af
Most confused POS tags


,count,percent
ADV,13.0,0.156627
DET,13.0,0.156627
X,11.0,0.132530
PROPN,11.0,0.132530
ADJ,8.0,0.096386


Most confused POS pairs


,pred,actual,count,percent
85,ADJ,DET,11.0,0.132530
203,X,PROPN,8.0,0.096386
44,PRON,ADV,6.0,0.072289
279,NOUN,X,4.0,0.048193
34,ADJ,ADV,4.0,0.048193


Possible tags for the word sodanige
{'DET': 11}
Possible tags for the word Crime
{'PROPN': 7}
Possible tags for the word hier
{'ADV': 9}
Possible tags for the word aanspreeklik
{'ADV': 2, 'ADJ': 2}
Possible tags for the word gebruik
{'VERB': 20, 'NOUN': 15}
Supervised
Relative mistaken word frequency


...

,% mistaken,all_count,mistaken_count
clanking,1.0,1,1.0
MCT,1.0,1,1.0
Mad,1.0,1,1.0
Least,1.0,1,1.0
Investment,1.0,1,1.0


Most mistaken words per target language from all source languages


is     1
the    1
all    1
's     1
to     1
dtype: int64

Most mistaken POS tags per target language from all source languages


DET     1
VERB    1
NOUN    1
dtype: int64

In [57]:
top1_mistaken_pos_per_target(most_mistaken_pos_lang)

Top 1 mistaken POS tag for all target languages


,lang,tag
0,af,ADV
1,ar,NOUN
2,en,NOUN


## Further analysis

We can use stored tables to zoom in on particular language pairs. Here we look at the predictions when Afrikaan was the source language and English was the target language.

In [53]:
mistakes_df = mistakes['af']['en']
# show examples when DET was the actual tag
mistakes_df[mistakes_df['actual']=='DET']

,predicted,actual,token,sentence,actual_tokens
20,PRON,DET,their,The Cunha report on multiannual guidance programmes comes before Parliament on Thursday and contains a proposal in paragraph 6 that a form of quota penalties should be introduced for countries which fail to meet their fleet reduction targets annually .,DET PROPN NOUN ADP ADJ NOUN NOUN VERB ADP NOUN ADP NOUN CCONJ VERB DET NOUN ADP NOUN NUM PRON DET NOUN ADP NOUN NOUN AUX AUX VERB ADP NOUN PRON VERB PART VERB DET ADJ NOUN NOUN ADV PUNCT
31,PRON,DET,another,"In many ways , the prerequisites differ from one Member State to another .",ADP ADJ NOUN PUNCT DET NOUN VERB ADP NUM NOUN NOUN ADP DET PUNCT
33,PRON,DET,his,The reason Mr Koch produced his sound report was because the work in the CEN and within the United Nations Economic Commission was proceeding none too expeditiously .,DET NOUN NOUN PROPN VERB DET ADJ NOUN VERB SCONJ DET NOUN ADP DET PROPN CCONJ ADP DET PROPN PROPN ADJ NOUN AUX VERB ADV ADV ADV PUNCT
42,PRON,DET,his,"Mr President , I would once again like to congratulate Mr Koch on his magnificent work on this other report , which in a way supplements the debate which we held in October on rail transport .",NOUN NOUN PUNCT PRON AUX ADV ADV VERB PART VERB NOUN PROPN ADP DET ADJ NOUN ADP DET ADJ NOUN PUNCT PRON ADP DET NOUN VERB DET NOUN PRON PRON VERB ADP PROPN ADP NOUN NOUN PUNCT
47,PRON,DET,its,We oppose the excessive control the central administration of the Union and its Member States exercises and we are calling for a reduction in the bureaucracy that has taken root in the drafting and implementation of programmes .,PRON VERB DET ADJ NOUN DET ADJ NOUN ADP DET PROPN CCONJ DET NOUN NOUN VERB CCONJ PRON AUX VERB ADP DET NOUN ADP DET NOUN PRON AUX VERB NOUN ADP DET NOUN CCONJ NOUN ADP NOUN PUNCT
57,PRON,DET,each,"as far as French planning experts are concerned , for example , the most probable scenario today is that of the entrenchment of regional disparities within each country .",ADP ADV ADP ADJ NOUN NOUN AUX ADJ PUNCT ADP NOUN PUNCT DET ADV ADJ NOUN ADV AUX PRON ADP DET NOUN ADP ADJ NOUN ADP DET NOUN PUNCT
59,PRON,DET,your,"In addition , if you choose to use Facebook from your mobile device , please note that you will be responsible for any fees associated with internet usage and / or text messaging as determined by your mobile service provider .",ADP NOUN PUNCT SCONJ PRON VERB PART VERB PROPN ADP DET ADJ NOUN PUNCT ADV VERB SCONJ PRON AUX AUX ADJ ADP DET NOUN VERB ADP NOUN NOUN CCONJ PUNCT CCONJ NOUN NOUN SCONJ VERB ADP DET NOUN NOUN NOUN PUNCT
62,PRON,DET,any,"In addition , if you choose to use Facebook from your mobile device , please note that you will be responsible for any fees associated with internet usage and / or text messaging as determined by your mobile service provider .",ADP NOUN PUNCT SCONJ PRON VERB PART VERB PROPN ADP DET ADJ NOUN PUNCT ADV VERB SCONJ PRON AUX AUX ADJ ADP DET NOUN VERB ADP NOUN NOUN CCONJ PUNCT CCONJ NOUN NOUN SCONJ VERB ADP DET NOUN NOUN NOUN PUNCT
65,PRON,DET,your,"In addition , if you choose to use Facebook from your mobile device , please note that you will be responsible for any fees associated with internet usage and / or text messaging as determined by your mobile service provider .",ADP NOUN PUNCT SCONJ PRON VERB PART VERB PROPN ADP DET ADJ NOUN PUNCT ADV VERB SCONJ PRON AUX AUX ADJ ADP DET NOUN VERB ADP NOUN NOUN CCONJ PUNCT CCONJ NOUN NOUN SCONJ VERB ADP DET NOUN NOUN NOUN PUNCT
69,PRON,DET,your,"To sign up for a brand new account , enter your name , birthday , gender , and email address into the form on www.facebook.com .",ADP VERB ADP ADP DET ADV ADJ NOUN PUNCT VERB DET NOUN PUNCT NOUN PUNCT NOUN PUNCT CCONJ NOUN NOUN ADP DET NOUN ADP PROPN PUNCT
